# Load pretrained resNet to predict species of plant

### Import libraries and required packages

#### descriptions
1. **pandas** : reading csv file.(e.g., ground-truth)
2. **torch** : load a model for prediction.
3. **os** : find out your directory path.
4. **cv2** : generate a numeric representation from your orginal image file

In [3]:
import pandas as pd
import torch
import os
import cv2

### Define the directory

#### define the path your model stays

In [2]:
CKPT_DIR = 'ckpt'
CKPT_PATH = os.path.join(CKPT_DIR, 'resNet_15_compose.ckpt')

#### define the ground-truth path

In [3]:
LABEL_DIR = 'label2.csv'
label_df = pd.read_csv(LABEL_DIR)
label_df.head()

,dirpath,target,label
0,DaanForestPark/20180330/1lZsRrQzj,通泉草,66
1,DaanForestPark/20180330/4PdXwYcGt,紫花酢漿草,51
2,DaanForestPark/20180330/6VrmeiUE5,通泉草,66
3,DaanForestPark/20180330/7nrSoiuHL,通泉草,66
4,DaanForestPark/20180330/7RKdkJLgy,通泉草,66


#### define your img file

In [4]:
DATA_SAMPLE = 'sample/sample.jpg'

### Image to RGB representaion function

In [5]:
def img2rgb(path):
    """
    :param path: String, location of the image file.
    :returns rgb-format numpy array.
    """
    img = cv2.imread(path)
    return cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

### Load your .ckpt model

In [6]:
model = torch.load(CKPT_PATH)

### Predict the result when an image comes

In [10]:
# convert image to rgb-format numpy array
oneshot = img2rgb(DATA_SAMPLE)
# do dimension transpose: (height, weidth, channel) to (channel, height, weidth)
oneshot = torch.Tensor(oneshot).permute(2,0,1).unsqueeze(0)
# use the cuda wrapper
oneshot = oneshot.cuda()
# input shape
print('shape of oneshot={}'.format(oneshot.shape))
pred = model(oneshot)
# output shape
print('shape of pred={}'.format(pred.shape))
# get a predicted label
_, predicted = torch.max(pred.cpu().data, 1)

shape of oneshot=torch.Size([1, 3, 192, 340])
shape of pred=torch.Size([1, 85])


In [11]:
res = label_df[label_df['label']==predicted.item()]['target'].unique()[0]
print('the predicted result is {}'.format(res))

the predicted result is 通泉草


In [7]:
input_names = ['whole_model.ckpt']
model = torch.load(input_names[0]) # map_location='cpu'
feature = torch.rand(1, 3, 224, 224).cuda()
for key, module in model._modules.items():
    input_names.append("l_{}_".format(key) + module._get_name())
torch_out = torch.onnx.export(model,feature,"model_best.onnx", export_params = True, verbose = True)

graph(%input.1 : Float(1, 3, 224, 224),
      %conv1.weight : Float(64, 3, 7, 7),
      %bn1.weight : Float(64),
      %bn1.bias : Float(64),
      %bn1.running_mean : Float(64),
      %bn1.running_var : Float(64),
      %bn1.num_batches_tracked : Long(),
      %layer1.0.conv1.weight : Float(64, 64, 3, 3),
      %layer1.0.bn1.weight : Float(64),
      %layer1.0.bn1.bias : Float(64),
      %layer1.0.bn1.running_mean : Float(64),
      %layer1.0.bn1.running_var : Float(64),
      %layer1.0.bn1.num_batches_tracked : Long(),
      %layer1.0.conv2.weight : Float(64, 64, 3, 3),
      %layer1.0.bn2.weight : Float(64),
      %layer1.0.bn2.bias : Float(64),
      %layer1.0.bn2.running_mean : Float(64),
      %layer1.0.bn2.running_var : Float(64),
      %layer1.0.bn2.num_batches_tracked : Long(),
      %layer1.1.conv1.weight : Float(64, 64, 3, 3),
      %layer1.1.bn1.weight : Float(64),
      %layer1.1.bn1.bias : Float(64),
      %layer1.1.bn1.running_mean : Float(64),
      %layer1.1.bn1.runnin

In [8]:
from onnx_coreml import convert
# Load the ONNX model as a CoreML model
model = convert(model='model_best.onnx',minimum_ios_deployment_target='13')
# Save the CoreML model
model.save('model_best.mlmodel')

1/69: Converting Node Type Conv
2/69: Converting Node Type BatchNormalization
3/69: Converting Node Type Relu
4/69: Converting Node Type MaxPool
5/69: Converting Node Type Conv
6/69: Converting Node Type BatchNormalization
7/69: Converting Node Type Relu
8/69: Converting Node Type Conv
9/69: Converting Node Type BatchNormalization
10/69: Converting Node Type Add
11/69: Converting Node Type Relu
12/69: Converting Node Type Conv
13/69: Converting Node Type BatchNormalization
14/69: Converting Node Type Relu
15/69: Converting Node Type Conv
16/69: Converting Node Type BatchNormalization
17/69: Converting Node Type Add
18/69: Converting Node Type Relu
19/69: Converting Node Type Conv
20/69: Converting Node Type BatchNormalization
21/69: Converting Node Type Relu
22/69: Converting Node Type Conv
23/69: Converting Node Type BatchNormalization
24/69: Converting Node Type Conv
25/69: Converting Node Type BatchNormalization
26/69: Converting Node Type Add
27/69: Converting Node Type Relu
28/69: